In [1]:
import plotly.graph_objects as go
from numpy import zeros, linalg


from func.main_for_4 import main_for_4
from func.Graph import Graph
import constants
from func.transition_matrix import transition_matrix
from func.matrix_1_4 import matrix_1_4

В '3. Transfer_matrix.ipynb' мы нашли точки пересечения тяг и рычагов. Теперь мы найдём углы **нижних** шаровых опор тяг.

Отрисуем предыдущий результат:

In [2]:
lower_points, upper_points, move_upper_CS, global_coordinate_system_upper_plate, local_coordinate_system_lower_point, local_coordinate_system_upper_point, arm_angle, global_cross_coordinates, matrix_move_lower_plate, local_cross_coordinates, matrix_move_upper_plate = main_for_4(
    25, 25, 300, 5, 5, 5)

fig = go.Figure()

graph = Graph(fig, lower_points, upper_points, move_upper_CS)
graph.upper_plate()
graph.lower_plate()

graph.coordinate_system(global_coordinate_system_upper_plate, 7, 1)
graph.coordinate_system_lower_plate()

counter = 1

for i in range(counter):
    graph.coordinate_system(local_coordinate_system_lower_point[i], i)
    graph.coordinate_system(local_coordinate_system_upper_point[i], i)
    graph.legs(i, f"group{i}", arm_angle, global_cross_coordinates)

graph.title()
graph.decor()

fig.show()

## 1 Нахождение матрицы перемещения для точек пересечения

Мы получили как глобальные, так и локальные координаты точек пересечения. Запишем локальные координаты в виде матриц линейного перемещения 4х4 и домножим на матрицы перемещения точек плит.

In [3]:
# Матрица перемещения для СК точек пересечения
matrix_move_crosspoints = zeros((6, 4, 4))

for i in range(counter):
    matrix_move_crosspoints[i] = matrix_move_lower_plate[i].dot(
                                        transition_matrix(local_cross_coordinates[i][0][0], local_cross_coordinates[i][1][0], local_cross_coordinates[i][2][0]))

Отрисуем локальную СК на точке пересечения

In [4]:
local_coordinate_crosspoint = zeros((6, 4, 4, 1))
global_coordinate_system = constants.GLOBAL_COORDINATE_SYSTEM

for j in range(6):
    for i in range(len(global_coordinate_system)):
        local_coordinate_crosspoint[j][i] = matrix_move_crosspoints[j].dot(matrix_1_4(global_coordinate_system[i][0],
                                                                    global_coordinate_system[i][1], 
                                                                    global_coordinate_system[i][2]))
        
for i in range(counter):
    graph.coordinate_system(local_coordinate_crosspoint[i], i)   

fig.show()         

## 2 Нахождение координат верхней шаровой опоры относительно нижней

На одной тяге 2 шаровых опоры, соответственно, нам нужно определить координаты одной опоры относительно другой в плоскости, в которой лежит рычаг (для нижней шаровой).

In [5]:
 # Коорд. верхней точки относительно пересечения
coordinates_upper_points_from_crosspoints = zeros((6, 4, 1))

for i in range(counter):
    # Находим координаты верхней шаровой относительно точек пересечения
    coordinates_upper_points_from_crosspoints[i] = linalg.inv(
                matrix_move_crosspoints[i]).dot(upper_points[i])

Мы получили локальные координаты верхней шаровой относительно нижней. Теперь нам нужно довернуть локальную СК так, чтобы плоскость ZX лежала на рычаге.

In [6]:
angles_between_crosspoints_and_upper_points = zeros(6)